In [1]:
## Load the model
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.messages import HumanMessage

query_text = 'how should I take care of my parents with Alzheimer disease?'
user_id = 'John Snow'

GENERATION_PROMPT = """
Based on the provided context, answer the user's question. If the context doesn't contain enough information or your not sure, say so.

Context:
{context}

Question: {question}

Return a clear and concise answer. The output should be in the following format:

Answer: <answer>
Sources: <sources>

If context isn't provided, use "Answer" should start with "Sorry, I don't have enough information to answer that question." and "Sources" should be empty.
"""

# memories = mem0.search(query_text, user_id)

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=GENERATION_PROMPT
)

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    HumanMessage(content=query_text),
    # ("system", prompt)
    
])

# query_text = 'what is the specialty for Chinese language. ' 

local_llm = 'marco-o1'
local_embedding_model = 'BAAI/bge-m3'

llm = ChatOllama(model=local_llm, temperature=0.0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0.0, format='json')

### Initialize Vector store and embedding model

Get peer support knowledge base and research knowledge base

---------------------------

In [2]:
from embedding.vector_store import VectorStore
from embedding.embedding_models import EmbeddingModels

embedding_model = EmbeddingModels().get_bge_embedding(model_name=local_embedding_model)
vectorstore = VectorStore()

peer_kb = vectorstore.get_chroma_vectorstore(
    vectorstore_path='./data/vector_database/peer_kb', 
    embedding=embedding_model
)

research_kb = vectorstore.get_chroma_vectorstore(
    vectorstore_path='./data/vector_database/research_kb', 
    embedding=embedding_model
)

2025-01-15 @ 12:31:24 | INFO     | utils.logger:<module>:56 - Logger initialized successfully


/Users/drakezhou/Development-Projects/LangchainRag_Ollama/venv/langchain_ollama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-01-15 @ 12:31:27 | INFO     | embedding.embedding_models:get_bge_embedding:28 - Using device: mps
2025-01-15 @ 12:31:27 | INFO     | embedding.embedding_models:get_bge_embedding:29 - Loading BGE embedding model: BAAI/bge-m3
2025-01-15 @ 12:31:31 | INFO     | embedding.embedding_models:get_bge_embedding:41 - Successfully loaded BGE embedding model


⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.
⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


In [4]:
from checkpoints.routering import get_routing_decision

intention_detection = get_routing_decision([HumanMessage(content=query_text)], model=local_llm)

2025-01-15 @ 12:31:57 | INFO     | checkpoints.routering:get_routing_decision:48 - Processing question: how should I take care of my parents with Alzheimer disease?
2025-01-15 @ 12:32:03 | INFO     | checkpoints.routering:get_routing_decision:64 - Routing decision: {'knowledge_base': 'research'}


In [6]:
similarity_search_res = vectorstore.retrieve_docs(query=query_text, vectorstore=peer_kb, k=5)

for i, doc in enumerate(similarity_search_res):
    print(f"---------------------------||Document Number: {i} ||------------------------ ")
    print(doc.page_content)
    print(doc.metadata)
    print("\n")

2025-01-15 @ 12:32:18 | WARNING  | embedding.vector_store:retrieve_docs:101 - Retrieved 5 documents
---------------------------||Document Number: 0 ||------------------------ 

You are not alone, twocat. We have several members on here that are taking care of family members that they had poor relationships with before Alzheimer's. It's a tough road for anyone, but your story has particular poignance. You may or may not have a legal obligation to care for your parent, and that may be something worth looking into, if for no other reason than knowing whether or not you "have to" do what you're doing.  Other people will have great advice and insight; I just want to make sure you get a response promptly. If you are able to get legal power of attorney and healthcare power of attorney those may be worthwhile, but only after you have discussed the situation, including prospects of keeping your mother where she is or moving her to your area and the legal ramifications of each. Alzheimer's is a 

### Retrieve grading 
*Preventing hallucination and erroneous retrieval that is not relevant to the question but based on the idiosyncrasies of the embedding model or chunking.*

---------------------

In [7]:
from typing import List
from checkpoints.retrieval_grading import grade_retrieval
from langchain.schema import Document

# Grade the retrieval results
graded_docs = grade_retrieval(query_text, similarity_search_res, model="llama3.2")

# Print graded results
# for doc in graded_docs:
#     print(f"Document content: {doc}")
    
# Filter out documents with relevance_score lower than 0.5
class ReasonedDocument:
    """
    A class to store the document, relevance score, and reasoning.
    """
    
    document: Document # Retrieved document from vector store in Document type
    relevance_score: float # Relevance score by reasoning model
    reasoning: str # Reasons for the relevance score
    
    def __init__(self, document, relevance_score, reasoning):
        self.document = document
        self.relevance_score = relevance_score
        self.reasoning = reasoning
        
    def __str__(self):
        return f"Document: {self.document}\nRelevance Score: {self.relevance_score}\nReasoning: {self.reasoning}\n"

# filtered_docs = [doc for doc, grade in zip(similarity_search_res, graded_docs) if grade["relevance_score"] >= 0.5]

filtered_docs = []

# Filter out documents with relevance_score lower than 0.5 and re-rank relevant documents based on relevance_score descending
filtered_docs = []

for doc, grade in zip(similarity_search_res, graded_docs):
    if grade["relevance_score"] >= 0.5:
        reasoned_doc = ReasonedDocument(
            document=doc,
            relevance_score=grade["relevance_score"],
            reasoning=grade["reasoning"]
        )
        filtered_docs.append(reasoned_doc)

# Re-rank the filtered documents based on relevance_score in descending order
filtered_docs.sort(key=lambda x: x.relevance_score, reverse=True)

# Print filtered results
for doc in filtered_docs:
    print(doc)
    # print(doc.metadata)



2025-01-15 @ 12:32:25 | INFO     | checkpoints.retrieval_grading:grade_retrieval:58 - Grading relevance for question: how should I take care of my parents with Alzheimer disease?
Document: page_content='=======

Many chapters of the Alzheimers Association have social workers who can sit down with families to talk productively and figure out care needs and expectations. Call your local chapter and see if they do care consults for families. I know multiple people who have had family members with differing views on care, finances, and responsibilities who found this very helpful. You need to be able to speak honestly and civilly about expectations and outcomes and also have realistic and accurate understanding of the disease progression mom will have. I would recommend you utilize a professional to facilitate, either through the Alz Assoc or someone you hire privately like a professional geriatric care manager. Is one of you DPOA? If not this stands to get quite messy and you will likely 

### Generate answer

*Using the graded documents to generate an answer.*

1. Only use the graded documents to generate an answer.

--------------------------------

In [ ]:
from answer_generation import generate_answer

context_chunks = [doc.document for doc in filtered_docs]

# TODO: handle the case where context_chunks is empty, answer should specify that we don't have relevant information.

answer = generate_answer(query_text, context_chunks, model="marco-o1")

print(answer)